# Assignment 1 - Exercise 3

**Riderlog**

In this notebook, we are going to use RiderLog dataset, which contains all bicycle trips collected by RiderLog between 2010 and 2013 in Greater Sydney region, thanks to [CityData](https://citydata.be.unsw.edu.au/layers/geonode%3AGreaterSydney_NSW_RiderLog_2010to2013).

The original data is in a point format, where bicycle trips are divided into points which are centroids of cells in a 500m * 500m grid. In order to better explore the relationship between attributes of routes, we kept only one records for each RouteId. While sacrificing some of the spatial information, the dataset is now more easy for you to play with!

*TODO: include how to transform the data from waypoints to routes*

Let's get started!

**First, lets import pandas library and the RiderLog data. **

**point_data** contains the original dataset, and the duplicates have been droped for you and stored in the variable **route_data**.

In [ ]:
import pandas as pd

point_data = pd.read_csv("https://citydata.be.unsw.edu.au/geoserver/wfs?typename=geonode%3AGreaterSydney_NSW_RiderLog_2010to2013&outputFormat=csv&version=1.0.0&request=GetFeature&service=WFS")

route_data = point_data.drop_duplicates(subset='RouteId') 
# Try printing out both point and route data to find out what does this line do.

Let's explore the dataset first!

__Question 1:__ How many routes are there in the dataset?

In [ ]:
route_data.shape

__Question 2:__ What is the mean, median and max value for distance?

__Question 3:__ Have you observed anything abnormal in the dataset? (*Hint: check the minimum values*)

In [ ]:
route_data.describe()

__Question 4:__ How many numerical attributes are there in the dataset? 

_Hint: Try get this information by getting the shape of the dataframe generated above by *describe* method._

In [ ]:
route_data.describe().shape

__Question 5:__ How many of them are integer attributes? <br />

*Hint*: Apart from describing the dataset, Python also knows how to count unique values! See [value_counts()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html) method.

In [ ]:
route_data.dtypes.value_counts()

Now let's see if we can findout relationships between attributes!


**Question 6:** How many records have a Duration larger than 60 mins?

Remember to tryout both the **shape** method and the new **value_counts** method!

In [ ]:
route_data[route_data['Duration'] > 60].shape

In [ ]:
(route_data['Duration'] > 60).value_counts()

**Question 7:**For all records matches the descrition above, what is the minimum distance of them?

Hint: To get information on a single column, you can **index that specific column** and perform something similar that you have used in the lecturer. Yes! I'm talking about max function. If you get stucked, try this [link](https://docs.python.org/3/library/functions.html)!

In [ ]:
min_distance_long_duration = min(route_data[route_data['Duration'] > 60]['Distance'])
print('Minimum distance for records which have a duration larger than 60 mins is', min_distance_long_duration)

You can also get this information by calling *describe* method on a subset of the route_data.

In [ ]:
route_data[route_data['Duration'] > 60].describe()

Have you noticed something abnormal? It seems that the rider didn't move a lot in that trip which lasted at least one hour!

To exclude these outliers, we need to clean the data before doing further analysis.

__Question 8:__ Generate a dataset contains only routes which have:
                1. an average travel speed between 50 km/h and 5 km/h;
                2. a duration longer than 60 minutes. 

To achieve this, let's try *loop* and *if* statements. (__This is not the best way to do it!__ But the purpose of this exercise is to get you familiar with loop and conditional statemets.)

The _drop_ method used in the code is explained [here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html).

In [ ]:
long_duration_routes = route_data[route_data['Duration'] > 60] #Generate a subset of route_data which meets condition 2.

print('The shape of the dataset before cleanning is: ', long_duration_routes.shape)

routes_to_be_dropped = [] # Keeping a record of rows to be dropped
for index, row in long_duration_routes.iterrows():
    if row.AvSpeed <= 3 or row.AvSpeed >= 50: 
        routes_to_be_dropped.append(index)
        
cleaned_dataset = long_duration_routes.drop(labels=routes_to_be_dropped)

print('The shape of the dataset after cleanning is: ', cleaned_dataset.shape)

**Question 9:** How many recreational trips have a duration longer than 60 minutes?

*Hint*:Use the dataset you cleaned above! Attribute *RidePurpos* denotes the purpose of the trip, and the key for recreational trips is *Recreation*.


In [ ]:
cleaned_dataset[cleaned_dataset['RidePurpos'] == 'Recreation'].shape

Now it's time for you to try on your own! Please use the _'cleaned dataset'_ instead of _'route data'_ from now on.

**Question 10:** What is the top speed for recreational trips with a female rider in cleaned dataset? 
The cell below shows a one-line yet illegible solution.

In [ ]:
max(cleaned_dataset[cleaned_dataset['RidePurpos'] == 'Recreation'][cleaned_dataset[cleaned_dataset['RidePurpos'] == 'Recreation']['Gender'] == 'F']['TopSpeed'])

Can you translate the code above into two or three lines of code so that people can better understanding your thinking?

Don't worry if you need to use more lines! Our goal is to make the code legible, which will make people's life easier.

In [ ]:
recreation_routes = cleaned_dataset[cleaned_dataset['RidePurpos'] == 'Recreation']
max(recreation_routes[recreation_routes['Gender'] == 'F']['TopSpeed'])

**Question 11:** What is the top 10 most popular destinations for recreational trips in cleaned dataset?

Hint: You can try to let the notebook print them all (and count manually), or you can make use of this _[nlargest](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.nlargest.html)_method

In [ ]:
cleaned_dataset['SA1_end'].value_counts().nlargest(n=10)

__Question 12:__ Let's make a scatter plots to see the distribution of the distance!

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode() # Plot data in the notebook not to a file

bined_distance = pd.cut(cleaned_dataset['Distance'], bins=20) #Distance are binned into 10 groups

# Use plotly to build a bar chart
trace = go.Bar(
    x=bined_distance.value_counts().index,  # The pandas series method value_counts() returns counts of unique values
    y=bined_distance.value_counts(),
    name='Distance Distribution'
)

# Define the layout
layout = go.Layout(title='Distance Distribution of cleaned dataset',
                   plot_bgcolor='rgb(230,230,230)', # Background colour in RGB (Red, Green, Blue) format
                   xaxis = dict(title='Bin Range'),  # Axis titles
                   yaxis = dict(title='Distance')
                  )

# Combine the trace and the layout into a figure
fig = go.Figure(data=[trace], layout=layout)

# Plot it!
iplot(fig)